Okey here is explained what i have doned. I have used pandas for storing the results.

The structure of the information: '.sqlite' files with one table named 'data' that contains all the tweets. The tweets has:
    - Index, unique id of the tweet.
    - Language, language of the tweet.
    - Location, location of the user according to his profile information.
    - List of the candidates (Arthaud, Asselineau, Cheminade, Dupont-Aignan, Filon, Hamon, Lassalle, Le-Pen, Macron, Melenchon, Poutou). The candidates have a 1 if the tweet is referred to them.
    - Original Tweet, the original tweet if is a retweeted tweet or corresponds to someone answering the tweet.
    - Original User, user that has written the original tweet.
    - Retweeted, 1 or 0 depending if the tweets is retweeted or not.
    - Tweet, text
    - Timestamp in milliseconds of the hour that the tweet has been written
    - User, name of the user
    - Day
    - This parameter i didnt know what it is but i have deleted it


First i have read all the '.sqlites' file and i have filter to obtain only the ones that are written in English, that is the condition in the first if structure.

Moreover, i have deleted some irrelevant data like the id of the tweet (because we have eliminate some of them), the language (in the English dataset) and the last paramater. I have pass the hour that was in miliseconds to normal datetime and renamed all the columns to have an idea of what structured our data is going to have.

For the location, it has been used the package GeoText for cleaning the location and split it in two different fields Country and City. Someone of them are imposible to clean so this two new fields has used the location parameter to full fill them.

I have used the vaderSentiment tool (you could install it via 'pip install vaderSentiment') and analyse the text to calculate the sentiment of each tweet. I have added this new column to the dataFrame and save the data in a .csv file.

In [1]:
import sqlite3 as lite

In [2]:
from os import scandir, getcwd

def ls(ruta = getcwd()+'/data/'):
    return [arch.name for arch in scandir(ruta) if arch.is_file()]

In [3]:
files = ls()
data_en = list()
all_data = list()
for file in files:
    con = lite.connect('data/'+file)

    cur = con.cursor()    
    cur.execute('SELECT * from data')

    data = cur.fetchall()
    for tweet in data:
        data_en.append(tweet)


In [4]:
from geotext import GeoText

def extract_city_country(location):
    city = ''
    country = ''
    places = GeoText(location.lower().title().strip())
    if len(places.cities) != 0:
        city = places.cities[0]
        if len(list(places.country_mentions.items())):
            if places.cities[0] == "Paris":
                country = "FR"
            else: country = list(places.country_mentions.items())[0][0]
    else:
        if len(list(places.country_mentions.items())):
            country = list(places.country_mentions.items())[0][0]
    return city,country
            

In [5]:
import re
def clean_jumplines(text):
    if text != None:
        text = re.sub('\n',' ',text)
        text = re.sub('\r',' ',text)
        text = re.sub('\t',' ',text)
        text = re.sub('\'',' ',text)
        text = re.sub('‘',' ',text)
        text = re.sub('’',' ',text)
        text = re.sub(';',' ',text)
    return text
    

In [6]:
data_final_en = list()

for tweet in data_en:
    if tweet[2]:
        city,country = extract_city_country(tweet[2])
        t = tweet + (city, country)
    else:
        t = tweet + ("","")

    data_final_en.append(t)
data_en = None

In [19]:
filter_tweets = list()
for tweet in data_final_en:
    if tweet[16] != '1':
        filter_tweets.append(tweet)


In [36]:
import time
import pandas
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()


jump = int(length/4)

for i in range(4):
    df = pandas.DataFrame(data=filter_tweets[(i)*jump:(i+1)*jump])
    del df[0]
    df[18] = df[18].apply(lambda x: time.strftime("%D %H:%M", time.localtime(int(x)/1000)).split()[1])
    del df[21] #delete
    df.columns = ['Language','Location', 'Anthaud', 'Asselineau', 'Cheminade', 'Dupont-Aignan', 'Fillon', 'Hamon', 'Lassalle', 'Le-Pen', 'Macron',
             'Melenchon', 'Poutou', 'Original-Tweet', 'Original-User', 'Retweeted', 'Tweet', 'Hour', 'User', 'Day', 'City', 'Country']
    df['Tweet'] = df['Tweet'].apply(clean_jumplines)
    df['Original-Tweet'] = df['Original-Tweet'].apply(clean_jumplines)
    df[df.columns] = df[df.columns].replace({',':' '}, regex=True)
    del df['Location']
    df.to_csv('elections_tweets_nosentiment'+str(i)+'.csv',sep=',')
    sentiment = list()

    for tweet in df['Tweet']:
            to_lang="en"
        from_lang=df['Language']
        if (from_lang == "en") or (from_lang == "en-US"):
            translation = sentence
            translator_name = "No translation needed"
        else: # please note usage limits for My Memory Translation Service:   http://mymemory.translated.net/doc/usagelimits.php
            # using   MY MEMORY NET   http://mymemory.translated.net
            api_url = "http://mymemory.translated.net/api/get?q={}&langpair={}|{}".format(sentence, from_lang, to_lang)
            hdrs ={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
                   'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
                   'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
                   'Accept-Encoding': 'none',
                   'Accept-Language': 'en-US,en;q=0.8',
                   'Connection': 'keep-alive'}
        response = requests.get(api_url, headers=hdrs)
        response_json = json.loads(response.text)
        translation = response_json["responseData"]["translatedText"]
        translator_name = "MemoryNet Translation Service"
        sentiment.append(analyzer.polarity_scores(transalation))
    real_sentiments = list()
    compound = list()
    for s in sentiment:
        if s['compound'] >= 0.5:
            real_sentiments.append('Positive')
        elif s['compound'] <= -0.5:
            real_sentiments.append('Negative')
        else:
            real_sentiments.append('Neutral')
        compound.append(s['compound'])
    df['Sentiment'] = pandas.Series(real_sentiments)
    df['Compound'] = pandas.Series(compound)
    df.to_csv('elections_tweets'+str(i)+'.csv',sep=',')